## DATA PREPARATION

### Libraries import and Spotify API authentication

In [7]:
# Import libraries
import numpy as np
import pandas as pd
import spotipy
import spotipy.util as util

In [8]:
# Authentication details
my_username = "oso41"
chosen_scope = "user-library-read"
my_client_id = ""
my_client_secret = ""

In [9]:
# Token creation
# Prompts a webpage where the user needs to authenticate. Then it redirect to a website, the url of which needs to be pasted
# below in order to validate the authorization

token = util.prompt_for_user_token(username=my_username,
                                   scope=chosen_scope,
                                   client_id=my_client_id,
                                   client_secret=my_client_secret,
                                   redirect_uri="http://127.0.0.1/")

In [10]:
# Create spotipy object
sp = spotipy.Spotify(auth=token)

### Functions to retrieve track features from a playlist

In [17]:
def get_tracks(playlist_id, playlist_length):
    '''Gets the main information for all tracks in a playlist
    
    Input:
    --------------
    pl_id (str): playlist id retrieved manually from spotify
    pl_length (int): number of tracks in the playlist
    
    Returns:
    --------------
    df (df): dataframe with tracks information (artist_name, track_name, track_id and popularity)
    '''
        
    artist_name = []
    track_name = []
    popularity = []
    track_id = []
    
    for i in range(0,playlist_length,100):
        results = sp.user_playlist_tracks(my_username,playlist_id=playlist_id,limit=100, offset=i)
        for i, t in enumerate(results['items']):
            artist_name.append(t['track']['artists'][0]['name'])
            track_name.append(t['track']['name'])
            track_id.append(t['track']['id'])
            popularity.append(t['track']['popularity'])
    
    df = pd.DataFrame([artist_name,track_name,track_id,popularity]).transpose()
    df.columns = ['artist_name','track_name','track_id','popularity']
    
    return df

In [18]:
def clean_tracks_df(df, like):
    '''Cleans dataframe and assigns new column with 1 if songs were liked or 0 if they were disliked
    
    Input:
    --------------
    df (df): dataframe with tracks information
    like (int): 1 if songs were liked or 0 if they were disliked
    
    Returns:
    --------------
    df (df): cleaned dataframe
    '''
    
    # Some songs appear twice (this is not avoided with unique uri), so they are deleted making sure 
    # that the combination of 'artist_name' and 'track_name' just appears once
    df = df.drop_duplicates(subset=['artist_name','track_name'])
    df = df.dropna()
    df['like'] = like

    return df

In [24]:
def get_features(df):
    '''Gets the features for all tracks in a dataframe
    
    Input:
    --------------
    df (df): dataframe with tracks information
    
    Returns:
    --------------
    df (df): dataframe with features for each track
    '''
    
    # Create empty list to store songs features
    rows = []

    # Retrieve audio features for every track
    for i in range(0,len(df['track_id']),50):
        feature_results = sp.audio_features(df['track_id'][i:i+50])
        for features in feature_results:
            rows.append(features)
    
    df = pd.DataFrame.from_dict(rows,orient='columns')
    
    return df  

In [25]:
def clean_features_df(df):
    '''Cleans dataframe containig tracks features
    
    Input:
    --------------
    df (df): dataframe with tracks features
    
    Returns:
    --------------
    df (df): cleaned dataframe
    '''
    
    # Drop columns containing useless information
    df = df.drop(['analysis_url','track_href','type','uri'],axis=1)
    # Change 'id' name to 'track_id' in order to match with previous dataframe (prepare for merging)
    df = df.rename(columns={'id':'track_id'})

    return df

In [32]:
def merge(df1,df2):
    '''Merges two dataframes with tracks information and tracks features
    
    Input:
    --------------
    df1 (df): dataframe with tracks information
    df2 (df): dataframe with features for each track
    
    Returns:
    --------------
    df (df): dataframe with tracks information and features for each track
    '''
    
    df = pd.merge(df1,df2,on='track_id',how='inner')
    
    return df

### Creation of good tracks dataframe

In [42]:
good_tracks = get_tracks('113sBEwMtYziAis9d3CmzJ',7000)

In [22]:
good_tracks_cl = clean_tracks_df(good_tracks,1)

In [23]:
good_tracks_cl.head()

,artist_name,track_name,track_id,popularity,like
0,Kyle Landry,Howl's Moving Castle Theme,3JTjiMAPVMfwjzPiG5R3hK,53,1
1,Kyle Landry,Shigatsu - Otouto Mitai Na Sonzai Piano,5tr1bCo0XRU3ZciXahlTKm,46,1
2,Kyle Landry,"Passion (From ""Kingdom Hearts II"") [Piano Solo]",2xFBrGSZ4mSURd8p5IU4ed,46,1
3,Kyle Landry,Dango Daikazoku,7c6mInVLlpnaYD55R72FDX,44,1
4,Kyle Landry,Interstellar - First Step,0kznmyiS92b9v1rkn9WDqV,40,1


In [27]:
good_features = get_features(good_tracks_cl)

In [29]:
good_features_cl = clean_features_df(good_features)

In [30]:
good_features_cl.head()

,acousticness,danceability,duration_ms,energy,track_id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.940,0.322,350348,0.1440,3JTjiMAPVMfwjzPiG5R3hK,0.917,7,0.0744,-17.448,0,0.0384,142.818,3,0.0471
1,0.967,0.368,302336,0.1240,5tr1bCo0XRU3ZciXahlTKm,0.803,3,0.1820,-21.851,1,0.0513,138.910,4,0.1040
2,0.964,0.406,274656,0.1300,2xFBrGSZ4mSURd8p5IU4ed,0.870,1,0.0964,-20.320,0,0.0515,116.005,4,0.0975
3,0.976,0.384,310974,0.0400,7c6mInVLlpnaYD55R72FDX,0.896,8,0.0943,-23.524,1,0.0357,95.854,4,0.1000
4,0.961,0.232,508622,0.0651,0kznmyiS92b9v1rkn9WDqV,0.930,9,0.0607,-21.866,0,0.0387,74.750,3,0.0372


In [33]:
good_df = merge(good_tracks_cl,good_features_cl)

In [34]:
good_df.head()

,artist_name,track_name,track_id,popularity,like,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Kyle Landry,Howl's Moving Castle Theme,3JTjiMAPVMfwjzPiG5R3hK,53,1,0.940,0.322,350348,0.1440,0.917,7,0.0744,-17.448,0,0.0384,142.818,3,0.0471
1,Kyle Landry,Shigatsu - Otouto Mitai Na Sonzai Piano,5tr1bCo0XRU3ZciXahlTKm,46,1,0.967,0.368,302336,0.1240,0.803,3,0.1820,-21.851,1,0.0513,138.910,4,0.1040
2,Kyle Landry,"Passion (From ""Kingdom Hearts II"") [Piano Solo]",2xFBrGSZ4mSURd8p5IU4ed,46,1,0.964,0.406,274656,0.1300,0.870,1,0.0964,-20.320,0,0.0515,116.005,4,0.0975
3,Kyle Landry,Dango Daikazoku,7c6mInVLlpnaYD55R72FDX,44,1,0.976,0.384,310974,0.0400,0.896,8,0.0943,-23.524,1,0.0357,95.854,4,0.1000
4,Kyle Landry,Interstellar - First Step,0kznmyiS92b9v1rkn9WDqV,40,1,0.961,0.232,508622,0.0651,0.930,9,0.0607,-21.866,0,0.0387,74.750,3,0.0372


### Creation of bad tracks dataframe

In [35]:
bad_tracks = get_tracks('2POKTRd37hqKGQS7hvV9Qo',3300)

In [36]:
bad_tracks_cl = clean_tracks_df(bad_tracks,0)

In [37]:
bad_tracks_cl.head()

,artist_name,track_name,track_id,popularity,like
0,Rkomi,Mon Cheri (feat. Sfera Ebbasta),66UkKyxN5YbJl9z7YamQOR,76,0
1,Priestess,Chef (feat. Madman),4QSLqqTFzsXR8MWfJeI3zq,59,0
2,Capo Plaza,Billets (feat. Ninho),2aD5ESGyEhXnXdoOsZvsR0,73,0
3,Sfera Ebbasta,Mademoiselle,2aPnpV1hT7l3eP10uSGK6c,80,0
4,MamboLosco,BINGO,2fXOb3Dr8di7hbcKFbYNPt,68,0


In [38]:
bad_features = get_features(bad_tracks_cl)

In [39]:
bad_features_cl = clean_features_df(bad_features)

In [40]:
bad_features_cl.head()

,acousticness,danceability,duration_ms,energy,track_id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.3160,0.887,181867,0.667,66UkKyxN5YbJl9z7YamQOR,0.0,6,0.1190,-5.681,0,0.0484,130.032,4,0.493
1,0.2770,0.944,176239,0.661,4QSLqqTFzsXR8MWfJeI3zq,0.0,4,0.2840,-5.643,0,0.0556,133.114,4,0.963
2,0.2980,0.732,198537,0.791,2aD5ESGyEhXnXdoOsZvsR0,0.0,1,0.1750,-5.642,1,0.1910,123.101,4,0.609
3,0.0671,0.794,189344,0.702,2aPnpV1hT7l3eP10uSGK6c,0.0,6,0.1550,-5.656,0,0.0346,121.938,4,0.535
4,0.1580,0.796,150444,0.643,2fXOb3Dr8di7hbcKFbYNPt,0.0,2,0.0561,-5.235,0,0.0354,128.057,4,0.609


In [41]:
bad_df = merge(bad_tracks_cl,bad_features_cl)

### Combine both dataframes

In [43]:
# MERGE GOOD AND BAD TRACKS

df_all = pd.concat([good_df,bad_df],axis=0,ignore_index=True)

In [47]:
df_all.shape

(9950, 18)

In [46]:
# There are five songs that I both liked and disliked
df_all['track_id'].nunique()

9945

In [48]:
# Remove these songs
df_all.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [50]:
df_all.shape

(9945, 18)

In [49]:
# Now all songs only appear once
df_all['track_id'].nunique()

9945

In [51]:
# Export to csv
df_all.to_csv('all_tracks.csv')

###  Create artists table

In [57]:
df_all['artist_name'].nunique()

3261

In [63]:
df_artists = pd.DataFrame(df_all['artist_name'].unique(),columns=['artist_name'])

In [65]:
df_artists.reset_index(inplace=True)

In [74]:
df_artists.rename(columns={'index':'artist_id'},inplace=True)

In [76]:
df_artists[df_artists['artist_name']=='Kyle Landry']['artist_id']

0    0
Name: artist_id, dtype: int64

In [79]:
df_artists.head()

,artist_id,artist_name
0,0,Kyle Landry
1,1,Cookin Soul
2,2,Trio Ternura
3,3,Gleemer
4,4,Black Peaks


In [80]:
df_all.head()

,artist_name,track_name,track_id,popularity,like,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Kyle Landry,Howl's Moving Castle Theme,3JTjiMAPVMfwjzPiG5R3hK,53,1,0.940,0.322,350348,0.1440,0.917,7,0.0744,-17.448,0,0.0384,142.818,3,0.0471
1,Kyle Landry,Shigatsu - Otouto Mitai Na Sonzai Piano,5tr1bCo0XRU3ZciXahlTKm,46,1,0.967,0.368,302336,0.1240,0.803,3,0.1820,-21.851,1,0.0513,138.910,4,0.1040
2,Kyle Landry,"Passion (From ""Kingdom Hearts II"") [Piano Solo]",2xFBrGSZ4mSURd8p5IU4ed,47,1,0.964,0.406,274656,0.1300,0.870,1,0.0964,-20.320,0,0.0515,116.005,4,0.0975
3,Kyle Landry,Dango Daikazoku,7c6mInVLlpnaYD55R72FDX,44,1,0.976,0.384,310974,0.0400,0.896,8,0.0943,-23.524,1,0.0357,95.854,4,0.1000
4,Kyle Landry,Interstellar - First Step,0kznmyiS92b9v1rkn9WDqV,40,1,0.961,0.232,508622,0.0651,0.930,9,0.0607,-21.866,0,0.0387,74.750,3,0.0372


In [81]:
df_final = df_all.merge(df_artists,how='outer',on='artist_name')

In [85]:
df_final.drop('artist_name',axis=1,inplace=True)

In [87]:
df_final.columns

Index(['track_name', 'track_id', 'popularity', 'like', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'artist_id'],
      dtype='object')

In [88]:
df_final = df_final[['artist_id','track_name', 'track_id', 'popularity', 'like', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'speechiness', 'tempo', 'time_signature', 'valence']]

In [90]:
df_final.head()

,artist_id,track_name,track_id,popularity,like,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0,Howl's Moving Castle Theme,3JTjiMAPVMfwjzPiG5R3hK,53,1,0.940,0.322,350348,0.1440,0.917,7,0.0744,-17.448,0,0.0384,142.818,3,0.0471
1,0,Shigatsu - Otouto Mitai Na Sonzai Piano,5tr1bCo0XRU3ZciXahlTKm,46,1,0.967,0.368,302336,0.1240,0.803,3,0.1820,-21.851,1,0.0513,138.910,4,0.1040
2,0,"Passion (From ""Kingdom Hearts II"") [Piano Solo]",2xFBrGSZ4mSURd8p5IU4ed,47,1,0.964,0.406,274656,0.1300,0.870,1,0.0964,-20.320,0,0.0515,116.005,4,0.0975
3,0,Dango Daikazoku,7c6mInVLlpnaYD55R72FDX,44,1,0.976,0.384,310974,0.0400,0.896,8,0.0943,-23.524,1,0.0357,95.854,4,0.1000
4,0,Interstellar - First Step,0kznmyiS92b9v1rkn9WDqV,40,1,0.961,0.232,508622,0.0651,0.930,9,0.0607,-21.866,0,0.0387,74.750,3,0.0372


### Export to database

In [52]:
from sqlalchemy import create_engine

In [53]:
ip = ''

login = ''

password = ''

db = 'spotify'

connection_string = f'mysql+pymysql://{login}:{password}@{ip}/{db}'

In [54]:
engine = create_engine(connection_string)

In [95]:
df_final.to_sql('spotify_tracks', con=engine, if_exists='replace')

In [96]:
df_artists.to_sql('spotify_artists', con=engine, if_exists='replace')